In [23]:
import ipywidgets as widgets
from IPython.display import display
import random
import string
from io import BytesIO

class UserInterface:
    def __init__(self):
        self.encrypted_data = ''
        self.decrypted_data = ''
        self.data_popup = widgets.Textarea(value='', placeholder='')
        self.encrypted_popup = widgets.Textarea(value='', placeholder='')
        self.decrypted_popup = widgets.Textarea(value='', placeholder='')
        self.setup_ui()
    
    def setup_ui(self):
        # Widgets for selecting encryption algorithm
        self.algorithm_dropdown = widgets.Dropdown(
            options=['Caesar Cipher', 'Atbash Cipher', 'Simple Substitution Cipher', 'Vigenère Cipher', 'Transposition Cipher', 'Affine Cipher'],
            description='Algorithm:',
            layout=widgets.Layout(width='400px')
        )

        # Button to randomly select an algorithm
        self.random_algorithm_button = widgets.Button(
            description='Random Algorithm',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
        self.random_algorithm_button.on_click(self.select_random_algorithm)

        # Text input for data
        self.data_input = widgets.Textarea(
            placeholder='Enter data to encrypt/decrypt',
            description='Data:',
            layout=widgets.Layout(width='400px', height='auto')
        )

        # Dropdown for key input method
        self.key_method_dropdown = widgets.Dropdown(
            options=['Manual Key Entry', 'Automatic Key Generation'],
            description='Key Method:',
            layout=widgets.Layout(width='400px')
        )

        # Text input for key
        self.key_input = widgets.Text(
            placeholder='Enter encryption/decryption key',
            description='Key:',
            layout=widgets.Layout(width='400px')
        )

        # Button for encryption
        self.encrypt_button = widgets.Button(
            description='Encrypt',
            button_style='success',
            layout=widgets.Layout(width='300px')
        )
        self.encrypt_button.on_click(self.encrypt)

        # Button for decryption
        self.decrypt_button = widgets.Button(
            description='Decrypt',
            button_style='warning',
            layout=widgets.Layout(width='300px')
        )
        self.decrypt_button.on_click(self.decrypt)

        # Button for saving encrypted data
        self.save_encrypted_button = widgets.Button(
            description='Save Encrypted Data',
            button_style='success',
            layout=widgets.Layout(width='300px')
        )
        self.save_encrypted_button.on_click(self.save_encrypted)

        # Button for saving decrypted data
        self.save_decrypted_button = widgets.Button(
            description='Save Decrypted Data',
            button_style='warning',
            layout=widgets.Layout(width='300px')
        )
        self.save_decrypted_button.on_click(self.save_decrypted)

        # Button for loading encrypted/decrypted data
        self.load_data_button = widgets.FileUpload(
            description='Load Encrypted/Decrypted Data',
            accept='.txt',  # Limit file type to text files
            multiple=False,  # Allow only one file to be uploaded
            button_style='danger',
            layout=widgets.Layout(width='300px', height='60px')
        )
        self.load_data_button.observe(self.load_data, names='value')
        
        # Slider for specifying number of encryption/decryption cycles
        self.cycle_slider = widgets.IntSlider(
            value=1,
            min=1,
            max=52,
            step=1,
            description='Levels:',
            layout=widgets.Layout(width='900px')
        )

        # Output widgets for displaying results
        self.encrypted_output = widgets.Textarea(
            value='',
            placeholder='Encrypted Data',
            description='Encryption Result:',
            layout=widgets.Layout(width='400px', height='auto'),
            disabled=True,  # Set disabled attribute to True
            style={'color': 'inherit', 'opacity': 1}  # Set style to maintain text color and opacity
        )

        self.decrypted_output = widgets.Textarea(
            value='',
            placeholder='Decrypted Data',
            description='Decryption Result:',
            layout=widgets.Layout(width='400px', height='auto'),
            disabled=True,  # Set disabled attribute to True
            style={'color': 'inherit', 'opacity': 1}  # Set style to maintain text color and opacity
        )

        # Create view buttons and assign them as instance variables
        self.view_data_button = widgets.Button(
            description='View Data',
            button_style='info',
            layout=widgets.Layout(width='150px', height='auto')
        )
        self.view_data_button.on_click(self.view_data)

        self.view_encrypted_button = widgets.Button(
            description='View Encrypted Data',
            button_style='info',
            layout=widgets.Layout(width='150px', height='auto')
        )
        self.view_encrypted_button.on_click(self.view_encrypted)

        self.view_decrypted_button = widgets.Button(
            description='View Decrypted Data',
            button_style='info',
            layout=widgets.Layout(width='150px', height='auto')
        )
        self.view_decrypted_button.on_click(self.view_decrypted)

        # Arrange buttons and slider in two columns
        left_column = widgets.VBox([self.encrypt_button, self.decrypt_button])
        right_column = widgets.VBox([self.save_encrypted_button, self.save_decrypted_button])
        right_right_column = widgets.VBox([self.load_data_button])

        # Arrange view buttons with respective fields
        view_data_group = widgets.HBox([self.data_input, self.view_data_button, self.data_popup], layout=widgets.Layout(width='100%', height='auto'))
        view_encrypted_group = widgets.HBox([self.encrypted_output, self.view_encrypted_button, self.encrypted_popup], layout=widgets.Layout(width='100%', height='auto'))
        view_decrypted_group = widgets.HBox([self.decrypted_output, self.view_decrypted_button, self.decrypted_popup], layout=widgets.Layout(width='100%', height='auto'))

        # Displaying widgets
        display(
            widgets.VBox([
                widgets.HBox([self.algorithm_dropdown, self.random_algorithm_button]), 
                view_data_group,
                self.key_method_dropdown,
                self.key_input,
                self.cycle_slider,
                widgets.HBox([left_column, right_column, right_right_column]), 
                view_encrypted_group,
                view_decrypted_group,
            ], layout=widgets.Layout(width='100%', height='auto'))
        )

        # Set up initial state of key method dropdown and key input based on selected algorithm
        self.update_key_widgets()

        # Define event handlers for dropdowns
        self.algorithm_dropdown.observe(self.on_algorithm_change, names='value')
        self.key_method_dropdown.observe(self.on_key_method_change, names='value')

    def view_data(self, _):
        self.update_popup(self.data_popup, self.data_input.value)

    def view_encrypted(self, _):
        self.update_popup(self.encrypted_popup, self.encrypted_output.value)

    def view_decrypted(self, _):
        self.update_popup(self.decrypted_popup, self.decrypted_output.value)

    def update_popup(self, popup, content):
        popup.value = content

    def on_algorithm_change(self, change):
        self.update_key_widgets()
    
    def on_key_method_change(self, change):
        self.update_key_widgets()
    
    def update_key_widgets(self):
        algorithm = self.algorithm_dropdown.value
        key_method = self.key_method_dropdown.value
        
        if algorithm in ['Caesar Cipher', 'Simple Substitution Cipher', 'Vigenère Cipher', 'Transposition Cipher', 'Affine Cipher']:
            self.key_method_dropdown.disabled = False
            self.key_input.disabled = key_method == 'Automatic Key Generation'
        else:
            self.key_method_dropdown.disabled = True
            self.key_input.disabled = True
            
    def select_random_algorithm(self, _):
        algorithms = self.algorithm_dropdown.options
        self.algorithm_dropdown.value = random.choice(algorithms)
    
    def encrypt(self, _):
        algorithm = self.algorithm_dropdown.value
        data = self.data_input.value
        key_method = self.key_method_dropdown.value
        key = self.key_input.value
        cycles = self.cycle_slider.value

        if not data:
            print("Error: Please enter data.")
            return

        if key_method == 'Automatic Key Generation':
            if algorithm == 'Simple Substitution Cipher':
                key = ''.join(random.sample(string.ascii_lowercase, len(string.ascii_lowercase)))
            elif algorithm == 'Caesar Cipher':
                key = str(random.randint(1, 25))
            elif algorithm == 'Vigenère Cipher':
                key = ''.join(random.choice(string.ascii_lowercase) for _ in range(len(data)))
            elif algorithm == 'Transposition Cipher':
                key = ''.join(random.sample(string.ascii_lowercase, random.randint(5, 10)))  # Random key length between 5 and 10
            elif algorithm == 'Affine Cipher':
                key = f"{random.randint(1, 25)},{random.randint(0, 25)}"
            self.key_input.value = key
        
        if algorithm != 'Atbash Cipher' and not key:
            print("Error: Please enter a key.")
            return

        try:
            result = data
            for _ in range(cycles):
                if algorithm == 'Caesar Cipher':
                    result = self.caesar_cipher_encrypt(result, int(key))
                elif algorithm == 'Atbash Cipher':
                    result = self.atbash_cipher_encrypt(result)
                elif algorithm == 'Simple Substitution Cipher':
                    if len(key) != 26:
                        print("Error: Key must be exactly 26 characters long for Simple Substitution Cipher.")
                        return
                    result = self.simple_substitution_cipher_encrypt(result, key)
                elif algorithm == 'Vigenère Cipher':
                    result = self.vigenere_cipher_encrypt(result, key)
                elif algorithm == 'Transposition Cipher':
                    result = self.transposition_cipher_encrypt(result, key)
                elif algorithm == 'Affine Cipher':
                    a, b = map(int, key.split(','))
                    result = self.affine_cipher_encrypt(result, a, b)
            
            self.encrypted_data = result
            self.encrypted_output.value = self.encrypted_data
            print("Encrypted data has been displayed in the text area below.")
        except ValueError as e:
            print("Error:", str(e))

    def decrypt(self, _):
        algorithm = self.algorithm_dropdown.value
        cycles = self.cycle_slider.value

        if not self.encrypted_data:
            print("Error: No data has been encrypted yet.")
            return
        try:
            result = self.encrypted_data
            for _ in range(cycles):
                if algorithm == 'Caesar Cipher':
                    result = self.caesar_cipher_decrypt(result, int(self.key_input.value))
                elif algorithm == 'Atbash Cipher':
                    result = self.atbash_cipher_decrypt(result)
                elif algorithm == 'Simple Substitution Cipher':
                    if len(self.key_input.value) != 26:
                        print("Error: Key must be exactly 26 characters long for Simple Substitution Cipher.")
                        return
                    result = self.simple_substitution_cipher_decrypt(result, self.key_input.value)
                elif algorithm == 'Vigenère Cipher':
                    result = self.vigenere_cipher_decrypt(result, self.key_input.value)
                elif algorithm == 'Transposition Cipher':
                    result = self.transposition_cipher_decrypt(result, self.key_input.value)
                elif algorithm == 'Affine Cipher':
                    a, b = map(int, self.key_input.value.split(','))
                    result = self.affine_cipher_decrypt(result, a, b)
            
            self.decrypted_data = result
            self.decrypted_output.value = self.decrypted_data
            print("Decrypted data has been displayed in the text area below.")
        except ValueError as e:
            print("Error:", str(e))

    def save_encrypted(self, _):
        if self.encrypted_data:
            filename = "encrypted_data.txt"
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(self.encrypted_data)
            print("Encrypted data saved to:", filename)
        else:
            print("Error: No data has been encrypted yet.")

    def save_decrypted(self, _):
        if self.decrypted_data:
            filename = "decrypted_data.txt"
            with open(filename, 'w') as file:
                file.write(self.decrypted_data)
            print("Decrypted data saved to:", filename)
        else:
            print("Error: No data has been decrypted yet.")

    def load_data(self, change):
        uploaded_file = self.load_data_button.value
        if uploaded_file:
            content = uploaded_file[list(uploaded_file.keys())[0]]['content']
            file_data = content.decode('utf-8')
            self.data_input.value = file_data
            print("Data loaded successfully.")
        else:
            print("Error: No file uploaded.")

    def caesar_cipher_encrypt(self, data, key):
        encrypted_data = ""
        for char in data:
            if char.isalpha():
                shifted = ord(char) + key
                if char.islower():
                    shifted = (shifted - ord('a')) % 26 + ord('a')
                elif char.isupper():
                    shifted = (shifted - ord('A')) % 26 + ord('A')
                encrypted_data += chr(shifted)
            else:
                encrypted_data += char
        return encrypted_data
    
    def caesar_cipher_decrypt(self, data, key):
        return self.caesar_cipher_encrypt(data, -key)
    
    def atbash_cipher_encrypt(self, data):
        encrypted_data = ""
        for char in data:
            if char.isalpha():
                if char.islower():
                    encrypted_data += chr(ord('a') + (ord('z') - ord(char)))
                elif char.isupper():
                    encrypted_data += chr(ord('A') + (ord('Z') - ord(char)))
            else:
                encrypted_data += char
        return encrypted_data
    
    def atbash_cipher_decrypt(self, data):
        return self.atbash_cipher_encrypt(data)
        
    # Simple Substitution Cipher
    def simple_substitution_cipher_encrypt(self, data, key):
        result = ''
        for char in data:
            if char.isalpha():
                shifted = key[ord(char) - 97] if char.islower() else key[ord(char) - 65]
                result += shifted
            else:
                result += char
        return result
    
    def simple_substitution_cipher_decrypt(self, data, key):
        reverse_key = ''.join(sorted(key))
        return self.simple_substitution_cipher_encrypt(data, reverse_key)
        
    def vigenere_cipher_encrypt(self, data, key):
        key_length = len(key)
        encrypted_data = ""
        for i, char in enumerate(data):
            if char.isalpha():
                shift = ord(key[i % key_length].lower()) - ord('a')
                if char.islower():
                    encrypted_data += chr(((ord(char) - ord('a') + shift) % 26) + ord('a'))
                elif char.isupper():
                    encrypted_data += chr(((ord(char) - ord('A') + shift) % 26) + ord('A'))
            else:
                encrypted_data += char
        return encrypted_data
    
    def vigenere_cipher_decrypt(self, data, key):
        key_length = len(key)
        decrypted_data = ""
        for i, char in enumerate(data):
            if char.isalpha():
                shift = ord(key[i % key_length].lower()) - ord('a')
                if char.islower():
                    decrypted_data += chr(((ord(char) - ord('a') - shift) % 26) + ord('a'))
                elif char.isupper():
                    decrypted_data += chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
            else:
                decrypted_data += char
        return decrypted_data
    
    def transposition_cipher_encrypt(self, data, key):
        key_len = len(key)
        sorted_key_indices = sorted(range(len(key)), key=lambda k: key[k])
        
        # Create empty columns
        columns = [''] * key_len
        
        # Fill the columns row-wise
        for i, char in enumerate(data):
            columns[i % key_len] += char
        
        # Read columns by key order
        encrypted_data = ''.join(columns[i] for i in sorted_key_indices)
        
        return encrypted_data
    
    def transposition_cipher_decrypt(self, data, key):
        key_len = len(key)
        data_len = len(data)
        sorted_key_indices = sorted(range(len(key)), key=lambda k: key[k])
        
        # Calculate the number of rows
        num_rows = data_len // key_len
        num_long_cols = data_len % key_len
        
        # Create empty columns
        columns = [''] * key_len
        
        # Determine the length of each column
        col_lengths = [num_rows + 1 if i < num_long_cols else num_rows for i in range(key_len)]
        
        # Fill the columns by sorted key order
        current_index = 0
        for i in sorted_key_indices:
            col_len = col_lengths[i]
            columns[i] = data[current_index:current_index + col_len]
            current_index += col_len
        
        # Read off the plaintext row-wise from the columns
        decrypted_data = ''
        for i in range(num_rows + 1):
            for col in columns:
                if i < len(col):
                    decrypted_data += col[i]
        
        return decrypted_data

    def affine_cipher_encrypt(self, data, a, b):
        encrypted_data = ''
        for char in data:
            if char.isalpha():
                base = ord('a') if char.islower() else ord('A')
                encrypted_data += chr(((a * (ord(char) - base) + b) % 26) + base)
            else:
                encrypted_data += char
        return encrypted_data

    def affine_cipher_decrypt(self, data, a, b):
        mod_inv = self.multiplicative_inverse(a, 26)
        decrypted_data = ''
        for char in data:
            if char.isalpha():
                base = ord('a') if char.islower() else ord('A')
                decrypted_data += chr(((mod_inv * ((ord(char) - base) - b)) % 26) + base)
            else:
                decrypted_data += char
        return decrypted_data

    def multiplicative_inverse(self, a, m):
        for i in range(1, m):
            if (a * i) % m == 1:
                return i
        raise ValueError(f"No multiplicative inverse for {a} modulo {m}")

# Create an instance of the UserInterface class
ui = UserInterface()